In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip glove.6B.zip

In [1]:
import json
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

In [ ]:
with open("./license_data_with_missing_parts.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
import spacy

nlp = spacy.blank("en")

sentences = []
labels = []
label_set = {"O"}

for item in data:
    text = item["text"]
    entities = item["entities"]

    doc = nlp(text)
    words = [token.text for token in doc]
    token_start_char_idxs = [token.idx for token in doc]

    label_seq = ["O"] * len(words)

    for entity in entities:
        ent_start = entity["start"]
        ent_end = entity["end"]
        ent_label = entity["label"]

        for i, token in enumerate(doc):
            token_start = token.idx
            token_end = token.idx + len(token)

            if token_start >= ent_start and token_end <= ent_end:
                if token_start == ent_start:
                    label_seq[i] = f"B-{ent_label}"
                else:
                    label_seq[i] = f"I-{ent_label}"
                label_set.add(label_seq[i])

    sentences.append(words)
    labels.append(label_seq)


In [ ]:
sentences[1]

In [ ]:
labels[1]